In [1]:
import pandas as pd
import numpy as np

kevin = pd.read_pickle('Download/Cleaned Data with Longformer/StoriesAboutKevin.csv.pkl')
creepy = pd.read_pickle('Download/Cleaned Data with Longformer/CreepyPasta.csv.pkl')
creepy = creepy[:len(kevin)].copy()

In [2]:
from textblob import TextBlob

In [3]:
zen = TextBlob(kevin.loc[4,'selftext'])

In [4]:
zen.sentences

[Sentence("I was living with my older brother while attending college and stayed up one night to watch the lunar eclipse."),
 Sentence("As I rushed outside my brother asked where I was going in such a rush."),
 Sentence("I told him I wanted to see the lunar eclipse."),
 Sentence("He the asked (and I quote), "Is that when the sun goes in front of the moon?""),
 Sentence("He wasn't always a Kevin."),
 Sentence("He was a hippie in the Sixties and just continued to drink and drug well past the point of idiocy."),
 Sentence("I've seen him nail his hand to a wall, walk off of a loading dock and face plant, stab himself in the knee with a screwdriver, loose the ability to read and write over time."),
 Sentence("I'm not sure a self-made Kevin counts but there it is.")]

In [5]:
for sentence in zen.sentences:
    print(sentence.sentiment)

Sentiment(polarity=0.16666666666666666, subjectivity=0.3333333333333333)
Sentiment(polarity=0.0, subjectivity=0.275)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=-0.275, subjectivity=0.325)
Sentiment(polarity=-0.07692307692307693, subjectivity=0.2692307692307692)
Sentiment(polarity=-0.25, subjectivity=0.8888888888888888)


In [6]:
zen.sentiment_assessments

Sentiment(polarity=-0.10146520146520145, subjectivity=0.38449328449328446, assessments=[(['older'], 0.16666666666666666, 0.3333333333333333, None), (['outside'], 0.0, 0.05, None), (['such'], 0.0, 0.5, None), (['past'], -0.25, 0.25, None), (['idiocy'], -0.3, 0.4, None), (['loose'], -0.07692307692307693, 0.2692307692307692, None), (['not', 'sure'], -0.25, 0.8888888888888888, None)])

In [7]:
tuple(list(zen.sentiment_assessments)[:2])

(-0.10146520146520145, 0.38449328449328446)

In [8]:
def blob_senti_assessments(text):
    zen = TextBlob(text)
    polarity, subjectivity = tuple(zen.sentiment_assessments)[:2]
    return polarity, subjectivity

creepy['polarity'], creepy['subjectivity'] = zip(*creepy['selftext'].map(blob_senti_assessments))
kevin['polarity'], kevin['subjectivity'] = zip(*kevin['selftext'].map(blob_senti_assessments))

In [9]:
import tensorflow as tf

In [10]:
creepyLF = tf.stack(creepy.loc[:,'LF pooler output'].to_list())
kevinLF = tf.stack(kevin.loc[:,'LF pooler output'].to_list())

creepyLF = tf.reshape(creepyLF, creepyLF.shape[::2])
kevinLF = tf.reshape(kevinLF, kevinLF.shape[::2])

In [11]:
import numpy as np
from sklearn.manifold import TSNE
import seaborn as sns

%matplotlib widget
import matplotlib.pyplot as plt

X_embedded_nosleep = TSNE(perplexity=30, learning_rate=50, n_components=3, n_iter=5000).fit_transform(creepyLF)
X_embedded_self = TSNE(perplexity=30, learning_rate=50, n_components=3, n_iter=2000).fit_transform(kevinLF)

df1 = pd.DataFrame(X_embedded_nosleep, columns=['x','y', 'z'])
df1['subreddit'] = 'creepy'
df1[['polarity','subjectivity']] = creepy[['polarity','subjectivity']]
df2 = pd.DataFrame(X_embedded_self, columns=['x','y', 'z'])
df2['subreddit'] = 'kevin'
df2[['polarity','subjectivity']] = kevin[['polarity','subjectivity']]

df = pd.concat([df1,df2])

In [12]:
df

,x,y,z,subreddit,polarity,subjectivity
0,-15.346008,-41.437752,-3.963685,creepy,0.500000,0.888889
1,-6.087790,-14.491609,9.949881,creepy,-0.156250,0.375000
2,-5.909782,-34.020397,18.488398,creepy,0.075000,0.400000
3,13.267905,-9.316761,24.250471,creepy,-0.026020,0.429932
4,-1.563671,-26.331141,3.544478,creepy,0.459615,0.532692
...,...,...,...,...,...,...
3398,34.205353,-11.127311,4.069194,kevin,-0.009167,0.492778
3399,-7.357478,-1.596703,21.046482,kevin,0.455000,0.280000
3400,4.808536,-12.250474,-2.052197,kevin,-0.247966,0.559127
3401,24.651768,-14.635349,-17.551437,kevin,-0.001716,0.413725


In [13]:
import matplotlib.cm as cm

creepy_colors = cm.rainbow(np.linspace(0, 1, 4))
kevin_colors = cm.rainbow(np.linspace(0, 1, 4))

In [14]:
df.T.values

array([[-15.34600830078125, -6.087789535522461, -5.909782409667969, ...,
        4.808536052703857, 24.65176773071289, 13.560359001159668],
       [-41.43775177001953, -14.491608619689941, -34.0203971862793, ...,
        -12.250473976135254, -14.63534927368164, -6.74772310256958],
       [-3.9636847972869873, 9.949880599975586, 18.4883975982666, ...,
        -2.052197217941284, -17.551437377929688, 1.373241901397705],
       ['creepy', 'creepy', 'creepy', ..., 'kevin', 'kevin', 'kevin'],
       [0.5, -0.15625, 0.07500000000000001, ..., -0.24796626984126982,
        -0.0017156862745098076, -0.08900793650793652],
       [0.8888888888888888, 0.375, 0.4, ..., 0.5591269841269841,
        0.41372549019607846, 0.2968650793650794]], dtype=object)

In [19]:
from mpl_toolkits.mplot3d import Axes3D,axes3d
sns.set(style = "darkgrid")



fig = plt.figure()
fig.set_size_inches(18.5, 10.5)
ax = fig.add_subplot(111, projection = '3d')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

ax.scatter(df[(df['polarity']>0) & (df['subjectivity']>0)]['x'], 
           df[(df['polarity']>0) & (df['subjectivity']>0)]['y'], 
           df[(df['polarity']>0) & (df['subjectivity']>0)]['z'], 
           c='brown', label='p&s > 0')
ax.scatter(df[(df['polarity']<0) & (df['subjectivity']>=0)]['x'], 
           df[(df['polarity']<0) & (df['subjectivity']>=0)]['y'], 
           df[(df['polarity']<0) & (df['subjectivity']>=0)]['z'], 
           c='blue', label='p<0, s>0', marker='^')
ax.scatter(df[(df['polarity']>=0) & (df['subjectivity']<0)]['x'], 
           df[(df['polarity']>=0) & (df['subjectivity']<0)]['y'], 
           df[(df['polarity']>=0) & (df['subjectivity']<0)]['z'], 
           c='yellow', label='p>0, s<0', marker='s')
ax.scatter(df[(df['polarity']<=0) & (df['subjectivity']<=0)]['x'], 
           df[(df['polarity']<=0) & (df['subjectivity']<=0)]['y'], 
           df[(df['polarity']<=0) & (df['subjectivity']<=0)]['z'], 
           c='green', label='p&s<0', marker='P')
plt.legend()

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …